In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)

In [ ]:
#Data from compustat:
forex_data = pd.read_csv('../../data/forex/compustat_forex.zip')

In [ ]:
forex_data.rename(columns={'datadate':'date', "curd" : "currency", "exratd_toUSD" : "to_usd"}, inplace=True)
forex_data = forex_data[["date", "currency", "to_usd"]]
forex_data["date"] = pd.to_datetime(forex_data["date"])

In [ ]:
forex_data.sort_values(["currency", "date"], inplace=True)
forex_data.reset_index(inplace=True, drop=True)

In [ ]:
forex_data["to_usd_12m"] = forex_data.groupby("currency")[["date", "to_usd"]].rolling("365D", on="date").mean().reset_index(drop=True)["to_usd"]

In [ ]:
for currency in forex_data["currency"].unique():
    min_date = forex_data[forex_data["currency"] == currency]["date"].min()
    forex_data.loc[(forex_data["currency"] == currency) & (forex_data["date"] < min_date + pd.DateOffset(days=365)), "to_usd_12m"] = np.nan

In [ ]:
forex_data[["currency", "date", "to_usd"]].dropna().to_parquet("../../data/forex/forex_data_daily.parquet", index=False)

In [ ]:
forex_data[["currency", "date", "to_usd_12m"]].dropna().to_parquet("../../data/forex/forex_data_12m.parquet", index=False)

In [ ]:
forex_data[["currency", "date", "to_usd", "to_usd_12m"]].dropna()